In [ ]:
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import START, StateGraph, MessagesState
from langgraph.prebuilt import tools_condition, ToolNode

In [ ]:
import os

from dotenv import load_dotenv
load_dotenv()
env = os.environ.copy()

from rich.console import Console
console = Console(width=96)

pubmed_server = "node_modules/@cyanheads/pubmed-mcp-server/dist/index.js"

from system_prompts import PUBMED_AGENT_PROMPT

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
client = MultiServerMCPClient(
    {
        "pubmed": {
            "command": "node",
            "args": [pubmed_server],
            "transport": "stdio",
            "env": env
        }
    }
)

In [ ]:
tools = await client.get_tools()

In [ ]:
for tool in tools:
    print(tool.name)

In [ ]:
tools

In [ ]:
# Find the tool by name
fetch_tool = tools[1]

# Manually invoke it to see the structure for a specific PMID
result = await fetch_tool.ainvoke({
    "pmids": ["31772153"],
    "detailLevel": "full_xml"
})

print(result)


In [ ]:
from pprint import pprint as pp
import json

# result[0] is the object you provided
# result[0].text contains the JSON string
pubmed_data = json.loads(result[0]['text'])

# The actual articles are in the "articles" list
articles = pubmed_data.get("articles", [])
# Print the first article found
if articles:
    console.print(json.dumps(articles[0], indent=2))


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

from system_prompts import (
    QUERY_PLANNER_AGENT,
    PUBMED_SEARCH_AGENT,
    PUBMED_FETCH_AGENT,
    METHODS_EXTRACTION_AGENT,
    METHODS_TO_JSON_AGENT,
    PMID_METHODS_FINAL_ASSEMBLY_AGENT,
)

# ---------------------------------------------------------------------
# LLM setup
# ---------------------------------------------------------------------

llm = ChatOpenAI(model="gpt-5-nano")
llm_with_tools = llm.bind_tools(tools)

# ---------------------------------------------------------------------
# Node helpers
# ---------------------------------------------------------------------

def llm_node(system_prompt: str, use_tools: bool = False):
    """
    Factory for simple LLM nodes with fixed system prompts.
    """
    sys_msg = SystemMessage(content=system_prompt)

    def _node(state: MessagesState):
        model = llm_with_tools if use_tools else llm
        response = model.invoke([sys_msg] + state["messages"])
        return {"messages": [response]}

    return _node

# ---------------------------------------------------------------------
# Nodes
# ---------------------------------------------------------------------

query_planner = llm_node(
    system_prompt=QUERY_PLANNER_AGENT,
    use_tools=False,
)

pubmed_search = llm_node(
    system_prompt=PUBMED_SEARCH_AGENT,
    use_tools=True,
)

pubmed_fetch = llm_node(
    system_prompt=PUBMED_FETCH_AGENT,
    use_tools=True,
)

methods_extraction = llm_node(
    system_prompt=METHODS_EXTRACTION_AGENT,
    use_tools=False,
)

methods_to_json = llm_node(
    system_prompt=METHODS_TO_JSON_AGENT,
    use_tools=False,
)

final_assembly = llm_node(
    system_prompt=PMID_METHODS_FINAL_ASSEMBLY_AGENT,
    use_tools=False,
)

# Tool execution node (shared)
tools_node = ToolNode(tools)

# ---------------------------------------------------------------------
# Build graph
# ---------------------------------------------------------------------

builder = StateGraph(MessagesState)

builder.add_node("query_planner", query_planner)
builder.add_node("pubmed_search", pubmed_search)
builder.add_node("pubmed_fetch", pubmed_fetch)
builder.add_node("methods_extraction", methods_extraction)
builder.add_node("methods_to_json", methods_to_json)
builder.add_node("final_assembly", final_assembly)
builder.add_node("tools", tools_node)

# ---------------------------------------------------------------------
# Edges
# ---------------------------------------------------------------------

builder.add_edge(START, "query_planner")
builder.add_edge("query_planner", "pubmed_search")

# Search → tool execution if needed
builder.add_conditional_edges(
    "pubmed_search",
    tools_condition,
)
builder.add_edge("tools", "pubmed_search")

# Fetch → tool execution if needed
builder.add_edge("pubmed_search", "pubmed_fetch")
builder.add_conditional_edges(
    "pubmed_fetch",
    tools_condition,
)
builder.add_edge("tools", "pubmed_fetch")

# Downstream processing (no tools)
builder.add_edge("pubmed_fetch", "methods_extraction")
builder.add_edge("methods_extraction", "methods_to_json")
builder.add_edge("methods_to_json", "final_assembly")
builder.add_edge("final_assembly", END)

# ---------------------------------------------------------------------
# Compile
# ---------------------------------------------------------------------

graph = builder.compile()

